In [2]:
# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from tqdm import tqdm

In [9]:
# --- CONFIGURAÇÕES ---
MARCAS = ['samsung', 'xiaomi', 'apple', 'motorola', 'realme', 'infinix', 'oppo', 'honor', 'asus']
ANOS_ALVO = ['2024', '2025', '2026'] 
PAGINAS = 3

In [ ]:


# --- CONEXÃO ---
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Referer': 'https://www.google.com/',
}
session.headers.update(headers)

# --- FUNÇÕES ---
def limpar_numero(texto):
    if not texto: return 0.0
    nums = re.findall(r'(\d+)', str(texto))
    return float(nums[-1]) if nums else 0.0

def limpar_preco(texto):
    if not texto: return 0.0
    nums = re.sub(r'[^\d,]', '', str(texto))
    try: return float(nums.replace(',', '.'))
    except: return 0.0

dados_finais = []

# --- VARREDURA SILENCIOSA ---
# Apenas uma barra de progresso para as marcas
pbar = tqdm(MARCAS, desc="Iniciando...", unit="marca")

for marca in pbar:
    # Atualiza a descrição da barra com a marca atual e quantos já pegou
    pbar.set_description(f"Lendo {marca.upper()} (Capturados: {len(dados_finais)})")
    
    for page in range(1, PAGINAS + 1):
        try:
            url_busca = f"https://www.oficinadanet.com.br/smartphones/buscar?palavra=&marca={marca}&pagina={page}"
            r = session.get(url_busca, timeout=20)
            soup = BeautifulSoup(r.content, 'html.parser')
            
            cards = soup.select('.list-item, .col-xl-3') 
            if not cards: cards = soup.select('a[href*="/smartphones/"]')
            if len(cards) == 0: break

            for card in cards:
                link_tag = card if card.name == 'a' else card.find('a', href=True)
                if not link_tag: continue
                
                url_produto = link_tag['href']
                if "/smartphones/" not in url_produto or "buscar" in url_produto: continue
                
                # Filtro Rápido (Card)
                texto_card = card.get_text(" ", strip=True)
                match_data = re.search(r'(\d{2}/\d{2}/(20\d{2}))', texto_card)
                
                ano_detectado = match_data.group(2) if match_data else None
                data_completa = match_data.group(1) if match_data else None
                
                if ano_detectado and ano_detectado not in ANOS_ALVO: continue 
                
                # Entrando no produto
                try:
                    r_prod = session.get(url_produto, timeout=15)
                    soup_prod = BeautifulSoup(r_prod.content, 'html.parser')
                    
                    h1 = soup_prod.find('h1')
                    nome = h1.get_text(strip=True).replace(': Ficha Técnica', '') if h1 else 'Desconhecido'
                    
                    if not data_completa:
                        infos = soup_prod.select('.obj-info-item')
                        for info in infos:
                            txt = info.get_text(" ", strip=True)
                            if "Lançamento" in txt:
                                match_interna = re.search(r'(\d{2}/\d{2}/(20\d{2}))', txt)
                                if match_interna:
                                    data_completa = match_interna.group(1)
                                    ano_detectado = match_interna.group(2)
                                    break
                    
                    if not ano_detectado or ano_detectado not in ANOS_ALVO: continue 
                    
                    # Extração
                    specs = {}
                    for el in soup_prod.select('.title-item, .title-cat'):
                        chave = el.get_text(strip=True).replace(":", "").strip()
                        val_div = el.find_next_sibling('div')
                        if val_div: 
                            valor = val_div.get_text(strip=True)
                            if valor and valor != "Não especificado": 
                                specs[chave] = valor
                    
                    # Monta Textão
                    texto_ia_lista = [f"Smartphone: {nome}", f"Marca: {marca.capitalize()}", f"Lançamento: {data_completa}"]
                    for k, v in specs.items(): texto_ia_lista.append(f"{k}: {v}")
                    texto_ia_final = ". ".join(texto_ia_lista) + "."

                    dados_finais.append({
                        'Nome': nome,
                        'Marca': marca.capitalize(),
                        'Ano': data_completa,
                        'Preco_Num': limpar_preco(specs.get('Preço atual')),
                        'RAM_Num': limpar_numero(specs.get('Memória RAM')),
                        'Armazenamento_Num': limpar_numero(specs.get('Armazenamento')),
                        'Bateria_Num': limpar_numero(specs.get('Bateria')),
                        'Specs_Completa': texto_ia_final,
                        'Url': url_produto
                    })
                    
                    # Atualiza o contador na barra em tempo real
                    pbar.set_description(f"Lendo {marca.upper()} (Capturados: {len(dados_finais)})")
                    time.sleep(0.05)
                    
                except Exception: pass
        except Exception: pass



Lendo ASUS (Capturados: 675): 100%|██████████| 9/9 [02:31<00:00, 16.87s/marca]    


✅ Concluído! 675 celulares salvos em 'dataset_celulares_final.csv'.


,Nome,Marca,Ano,Preco_Num,RAM_Num,Specs_Completa,Armazenamento_Num,Bateria_Num,Url
0,Samsung Galaxy Z TriFold,Samsung,01/12/2025,0.00,16.0,Smartphone: Samsung Galaxy Z TriFold. Marca: S...,1.0,5600.0,https://www.oficinadanet.com.br/smartphones/sa...
1,Samsung Galaxy A17 4G,Samsung,08/09/2025,1198.00,8.0,Smartphone: Samsung Galaxy A17 4G. Marca: Sams...,256.0,5000.0,https://www.oficinadanet.com.br/smartphones/sa...
2,Samsung Galaxy S25 FE,Samsung,04/09/2025,3098.00,8.0,Smartphone: Samsung Galaxy S25 FE. Marca: Sams...,512.0,4900.0,https://www.oficinadanet.com.br/smartphones/sa...
3,Samsung Galaxy A07 4G,Samsung,23/08/2025,646.94,8.0,Smartphone: Samsung Galaxy A07 4G. Marca: Sams...,256.0,5000.0,https://www.oficinadanet.com.br/smartphones/sa...
4,Samsung Galaxy A17 5G,Samsung,06/08/2025,1249.00,8.0,Smartphone: Samsung Galaxy A17 5G. Marca: Sams...,256.0,5000.0,https://www.oficinadanet.com.br/smartphones/sa...


In [ ]:
# --- SALVAR ---
if dados_finais:
    df = pd.DataFrame(dados_finais)
    
    cols_order = ['Nome', 'Marca', 'Ano', 'Preco_Num', 'RAM_Num', 'Specs_Completa']
    cols = [c for c in cols_order if c in df.columns] + [c for c in df.columns if c not in cols_order]
    df = df[cols]
    
    df.to_csv("dataset_celulares_final.csv", index=False)
    print(f"\n✅ Concluído! {len(df)} celulares salvos em 'dataset_celulares_final.csv'.")
    display(df.head())
else:
    print("\n❌ Nada encontrado.")

In [11]:
celulares_df = pd.read_csv("/home/caua/GoogleDrive/ENGENHARIA/COMPUTAÇÃO/MATÉRIAS SEMESTRE 2/PROJETOS 2/AMBIENTE/dataset_celulares_final.csv")

In [16]:
display(celulares_df.head())

,Nome,Marca,Ano,Preco_Num,RAM_Num,Specs_Completa,Armazenamento_Num,Bateria_Num,Url
0,Samsung Galaxy Z TriFold,Samsung,01/12/2025,0.00,16.0,Smartphone: Samsung Galaxy Z TriFold. Marca: S...,1.0,5600.0,https://www.oficinadanet.com.br/smartphones/sa...
1,Samsung Galaxy A17 4G,Samsung,08/09/2025,1198.00,8.0,Smartphone: Samsung Galaxy A17 4G. Marca: Sams...,256.0,5000.0,https://www.oficinadanet.com.br/smartphones/sa...
2,Samsung Galaxy S25 FE,Samsung,04/09/2025,3098.00,8.0,Smartphone: Samsung Galaxy S25 FE. Marca: Sams...,512.0,4900.0,https://www.oficinadanet.com.br/smartphones/sa...
3,Samsung Galaxy A07 4G,Samsung,23/08/2025,646.94,8.0,Smartphone: Samsung Galaxy A07 4G. Marca: Sams...,256.0,5000.0,https://www.oficinadanet.com.br/smartphones/sa...
4,Samsung Galaxy A17 5G,Samsung,06/08/2025,1249.00,8.0,Smartphone: Samsung Galaxy A17 5G. Marca: Sams...,256.0,5000.0,https://www.oficinadanet.com.br/smartphones/sa...


In [19]:
contagem = celulares_df["Marca"].value_counts()
print(contagem)
print(celulares_df.info())

Marca
Xiaomi      156
Motorola    123
Realme      117
Samsung     108
Honor        54
Oppo         51
Apple        27
Infinix      27
Asus         12
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675 entries, 0 to 674
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Nome               675 non-null    object 
 1   Marca              675 non-null    object 
 2   Ano                675 non-null    object 
 3   Preco_Num          675 non-null    float64
 4   RAM_Num            675 non-null    float64
 5   Specs_Completa     675 non-null    object 
 6   Armazenamento_Num  675 non-null    float64
 7   Bateria_Num        675 non-null    float64
 8   Url                675 non-null    object 
dtypes: float64(4), object(5)
memory usage: 47.6+ KB
None


'Smartphone: Samsung Galaxy Z TriFold. Marca: Samsung. Lançamento: 01/12/2025. Status: Lançamento. Data lançamento: 01/12/2025. Outras informações: Dimensões. Dimensões: Aberto: 159.2 x 214.1 x 3.9-4.2 mm e Fechado: 159.2 x 75 x 12.9 mm. Peso: 309g. Construção: Vidro Frontal (Gorilla Glass Victus Ceramic 2) Tela externa e Frame de alumínio. Chip: Multi eSIM e Nano-SIM. Proteção Água: IP48. Dobrável: Sim. Plataforma: Sistema. Sistema: Android 16 e One UI 8. Atualizações: 7 anos (sistema). Processador: Qualcomm Snapdragon 8 Elite (3 nm). CPU: Octa-core (2x4.47 GHz Oryon V2 Phoenix L, 6x3.53 GHz Oryon V2 Phoenix M). GPU: Adreno 830. Memórias: Memória RAM. Memória RAM: 16 GB. Tipo de Memória RAM: UFS 4.0. Armazenamento: 512 GBe1 TB. Armazenamento Extra: Não Possui. Tela: Tela - Tipo. Tela - Tipo: Tri-foldable Dynamic LTPO AMOLED 2X. Tela - Tamanho: 10 polegadas. Tela - Resolução: 1584 x 2160 pixels. Tela - Frequência: 120Hz. Tela - Densidade: 269 ppi. Tela - Proteção: Corning Gorilla Glass Victus 2. Tela - Extras: Tela externa 6.5 pol 1080 x 2520px, 1600 nits (pico) e HDR 10 Plus. Câmera principal: 200 MP, f/1.7, (wide), multi-directional PDAF e OIS. 2ª câmera: 10 MP, f/2.4, (telefoto), 3x zoom ótico, PDAF e OIS. 3ª câmera: 12 MP, f/2.2, 123˚, 12mm (ultrawide). Câmera - Extras: Panorama, HDR e Flash LED. Vídeo: 4K - 60fps, gyro-EIS e FULLHD - 60fps. Câmera frontal: Câm. Selfie. Câm. Selfie: Câmera Cover: 10 MP, f/2.2 e 10 MP, f/2.2, 24mm (wide). Câmera Frontal - Extras: HDR. Bateria: 5600 mAh. Carregador: 45W e 4.5 reverse wireless. Carrega sem fios?: 15W. Redes de dados: 3G. 3G: Sim. 4G: Sim. 5G: Sim. Wi-Fi: Wi-Fi 802.11 a/b/g/n/ac/6e/7, Tri-Band e Wi-Fi Direct. Bluetooth: 5.4, A2DP, LE e aptX HD. GPS: GPS, QZSS, BDS, GALILEO e GLONASS. NFC: Sim. Som: Loudspeaker e Estéreo. 3.5mm jack: Não. Outros: Rádio FM. Rádio FM: Não. Infravermelho: Não. USB: On-The-Go, Tipo-C e 3.2. Sensores: Impressão digital (lateral), Samsung DeX, Bússola, Proximidade, Barômetro, Giroscópio, Acelerômetro e Suporte ultra wideband (UWB).'


Utilizando o modelo

In [ ]:
from sentence_transformers import SentenceTransformer, util

# --- PEÇA 1: O MODELO (O TRADUTOR) ---
model = SentenceTransformer('all-MiniLM-L6-v2') 

# Aqui a IA trabalha. Ela transforma texto em vetor.
vetor_banco = model.encode(df['Specs_Completa']) 
vetor_usuario = model.encode("celular barato")

# --- PEÇA 2: O ALGORITMO (O MATEMÁTICO) ---
# Aqui NÃO TEM IA. É pura álgebra linear (multiplicação de matrizes).
# 'util.cos_sim' é a função que calcula o ângulo.
scores = util.cos_sim(vetor_usuario, vetor_banco)